## Library

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.models as models

import numpy as np
import pydicom
import os
import pandas as pd
from skimage.transform import resize
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

## Init GPU

In [ ]:
# Initialize GPU Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

print(device)

In [ ]:
%load_ext autoreload
%autoreload 2

## Config Info

In [ ]:
# Constants
TEST_SIZE = 0.2
HEIGHT = 224
WIDTH = 224
CHANNELS = 3
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
TEST_BATCH_SIZE = 2
SHAPE = (HEIGHT, WIDTH, CHANNELS)

# Folders
DATA_DIR = './rsna-mil-training'

In [ ]:
def preprocess_slice(slice, target_size=(224, 224)):
    slice = resize(slice, target_size, anti_aliasing=True)
    brain_channel = apply_windowing(slice, window=(40, 80))
    subdural_channel = apply_windowing(slice, window=(80, 200))
    bone_channel = apply_windowing(slice, window=(600, 2800))
    
    multichannel_slice = np.stack([brain_channel, subdural_channel, bone_channel], axis=-1)
    return multichannel_slice

def apply_windowing(slice, window):
    window_width, window_level = window
    lower_bound = window_level - window_width // 2
    upper_bound = window_level + window_width // 2
    
    windowed_slice = np.clip(slice, lower_bound, upper_bound)
    windowed_slice = (windowed_slice - lower_bound) / (upper_bound - lower_bound)
    return windowed_slice

In [ ]:
def read_dicom_folder(folder_path):
    slices = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".dcm"):
            file_path = os.path.join(folder_path, filename)
            ds = pydicom.dcmread(file_path)
            slices.append(ds.pixel_array)
    
    # Add black images if the number of slices is less than 60
    num_slices = len(slices)
    if num_slices < 60:
        black_slice = np.zeros_like(slices[0])
        for _ in range(60 - num_slices):
            slices.append(black_slice)
    
    return slices

In [ ]:
def process_patient_data(data_dir, row):
    """
    Process data for a single patient based on the row from the DataFrame.
    
    Args:
        data_dir (str): The directory containing DICOM folders.
        row (pd.Series): A row from the patient_scan_labels DataFrame.

    Returns:
        Tuple: Preprocessed slices and label.
    """
    patient_id = row['patient_id'].replace('ID_', '')  # Remove 'ID_' prefix
    study_instance_uid = row['study_instance_uid'].replace('ID_', '')  # Remove 'ID_' prefix
    
    # Construct folder path based on patient_id and study_instance_uid
    folder_name = f"{patient_id}_{study_instance_uid}"
    folder_path = os.path.join(data_dir, folder_name)
    
    # Read and preprocess DICOM slices
    if os.path.exists(folder_path):
        slices = read_dicom_folder(folder_path)
        preprocessed_slices = [preprocess_slice(slice) for slice in slices]
        
        # Determine label based on any of the hemorrhage indicators
        label = 1 if row[['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']].any() else 0
        
        return preprocessed_slices, label
    else:
        print(f"Folder not found: {folder_path}")
        return None, None  # Handle the case where the folder is not found

## Split Dataset

In [ ]:
def split_dataset(patient_scan_labels, test_size=0.2, random_state=42):
    """
    Split the dataset into training and testing sets while maintaining the same ratio of labels.

    Args:
        patient_scan_labels (pd.DataFrame): The DataFrame containing patient scan labels.
        test_size (float): The proportion of the dataset to include in the test split.
        random_state (int): The seed used by the random number generator.

    Returns:
        Tuple: train_labels, test_labels
    """
    # If exists of the hemorrhage indicators is 1, the label is 1, otherwise 0
    patient_scan_labels['label'] = patient_scan_labels[['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']].any(axis=1).astype(int)

    # Extract the labels from the DataFrame
    labels = patient_scan_labels['label']

    # Split the dataset into training and testing sets while maintaining the same ratio of labels
    train_labels, test_labels = train_test_split(patient_scan_labels, test_size=test_size, stratify=labels, random_state=random_state)

    return train_labels, test_labels


In [ ]:
class TrainDatasetGenerator(Dataset):
    """
    A custom dataset class for training data.
    """
    def __init__(self, data_dir, patient_scan_labels):
        self.data_dir = data_dir
        self.patient_scan_labels = patient_scan_labels

    def __len__(self):
        return len(self.patient_scan_labels)

    def __getitem__(self, idx):
        row = self.patient_scan_labels.iloc[idx]
        preprocessed_slices, label = process_patient_data(self.data_dir, row)
        
        if preprocessed_slices is not None:
            # Convert the list of numpy arrays to a single numpy array
            preprocessed_slices = np.array(preprocessed_slices)  # Convert to numpy array
            return torch.tensor(preprocessed_slices, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
        else:
            return None, None  # Handle the case where the folder is not found

class TestDatasetGenerator(Dataset):
    """
    A custom dataset class for testing data.
    """
    def __init__(self, data_dir, patient_scan_labels):
        self.data_dir = data_dir
        self.patient_scan_labels = patient_scan_labels

    def __len__(self):
        return len(self.patient_scan_labels)

    def __getitem__(self, idx):
        row = self.patient_scan_labels.iloc[idx]
        preprocessed_slices, label = process_patient_data(self.data_dir, row)
        
        if preprocessed_slices is not None:
            # Convert the list of numpy arrays to a single numpy array
            preprocessed_slices = np.array(preprocessed_slices)  # Convert to numpy array
            return torch.tensor(preprocessed_slices, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
        else:
            return None, None  # Handle the case where the folder is not found
        

# Function to create DataLoader for training
def get_train_loader(data_dir, patient_scan_labels, batch_size=TRAIN_BATCH_SIZE, shuffle=True):
    train_dataset = TrainDatasetGenerator(data_dir, patient_scan_labels)
    return DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

# Function to create DataLoader for testing
def get_test_loader(data_dir, patient_scan_labels, batch_size=TEST_BATCH_SIZE):
    test_dataset = TestDatasetGenerator(data_dir, patient_scan_labels)
    return DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## CNN Feature Extractor

In [ ]:
class FeatureExtractor(torch.nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        resnet18 = models.resnet18(pretrained=True)
        self.features = torch.nn.Sequential(*list(resnet18.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x.view(x.size(0), -1)

In [ ]:
class MILModelWithBiLSTM(nn.Module):
    def __init__(self, num_classes):
        super(MILModelWithBiLSTM, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.bilstm = nn.LSTM(input_size=512, hidden_size=128, num_layers=1, bidirectional=True)
        self.classifier = nn.Linear(256, num_classes)  # 128 * 2 for bidirectional

    def forward(self, x):
        batch_size, num_images, height, width, channels = x.size()
        x = x.view(-1, channels, height, width)
        features = self.feature_extractor(x)
        features = features.view(batch_size, num_images, -1)  # (batch_size, num_images, feature_dim)

        # Pass through BiLSTM
        lstm_out, _ = self.bilstm(features)
        
        # Use the last output for classification
        output = self.classifier(lstm_out[:, -1])
        
        return output

In [ ]:
data_dir = '../rsna-mil-training'
patient_scan_labels = pd.read_csv('data_analyze/training_1000_scan_subset.csv')

In [ ]:
train_labels, test_labels = split_dataset(patient_scan_labels, test_size=TEST_SIZE)

train_loader = get_train_loader(data_dir, train_labels, batch_size=TRAIN_BATCH_SIZE)
test_loader = get_test_loader(data_dir, test_labels, batch_size=TEST_BATCH_SIZE)

In [ ]:
from torch.cuda.amp import GradScaler, autocast
scaler = GradScaler()

num_epochs = 1
model = MILModelWithBiLSTM(num_classes=1).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    print('############################################')
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    total_loss = 0
    correct = 0
    total = 0
    
    model.train()

    train_loader_tqdm = tqdm(train_loader, leave=False)
    for batch_idx, (batch_slices, batch_labels) in enumerate(train_loader_tqdm):
        try:
            # Move data to device
            batch_slices = batch_slices.to(device)
            batch_labels = batch_labels.to(device)

            # Assert shapes for debugging
            assert batch_slices.ndim == 5, f"Expected 5D input, got {batch_slices.ndim}D"
            assert batch_labels.ndim == 1, f"Expected 1D labels, got {batch_labels.ndim}D"
            assert batch_slices.shape[0] == batch_labels.shape[0], "Batch size mismatch between slices and labels"
            
            optimizer.zero_grad()
            # outputs = model(batch_slices)
            # 
            # # Assert output shape for debugging
            # assert outputs.ndim == 2, f"Expected 2D output from model, got {outputs.ndim}D"
            # assert outputs.shape[0] == batch_labels.shape[0], "Output size mismatch with labels"
            # 
            # # Calculate loss
            # loss = criterion(outputs.squeeze(), batch_labels.float())
            # 
            # loss.backward()
            # optimizer.step()
            with autocast():  # Enable mixed precision
                outputs = model(batch_slices)
                loss = criterion(outputs.squeeze(), batch_labels.float())
            
                scaler.scale(loss).backward()  # Scale the loss for backpropagation
                scaler.step(optimizer)          # Update parameters
                scaler.update()                 # Update the scale for next iteration

            total_loss += loss.item()
            predicted = (outputs.squeeze() >= 0.5).float()
            correct += (predicted == batch_labels.float()).sum().item()
            total += batch_labels.size(0)

            batch_accuracy = (predicted == batch_labels.float()).float().mean().item()

            train_loader_tqdm.set_description(f"Epoch [{epoch + 1}/{num_epochs}]")
            train_loader_tqdm.set_postfix(loss=loss.item(), accuracy=batch_accuracy)

        except Exception as e:
            print(f"Error in batch {batch_idx + 1}: {str(e)}")
            print(f"Input shape: {batch_slices.shape}")
            print(f"Labels shape: {batch_labels.shape}")
            raise  # Re-raise the exception for full traceback

    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')